# Escalas armatore - Preparação dos dados

## Módulos

In [17]:
import sys

In [2]:
# from .env
sys.path.append('../.env/lib/python3.6/site-packages/')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import pingouin as pg

## Dados

In [5]:
#dados = pd.read_excel("../data/Planilha LIMPA PARA USO - Copia.xlsx", engine='openpyxl')
dados = pd.read_excel("../data/Planilha LIMPA PARA USO - Copia.xlsx")

In [6]:
dados_pca_score = pd.read_pickle('../data/dados_pca_score.pkl')

## Ajustar dados

- Colunas

In [7]:
dados.rename(columns = {'Codigo para SPSS':'idSpss',
                        'Est_Civil':'EstCivil',
                        'LAT3 (NORMAL)':'LAT3',
                        'LAT4 (NORMAL)':'LAT4'}, inplace = True)

- Preparar dados

In [8]:
dados_testes = dados.merge(dados_pca_score, on = 'idSpss', how = 'left')

In [9]:
dados_testes.head()

,idSpss,CD_Distribuicao,Estado,Pais,Cidade,Marca_Pre_Pos,Marc_Pref,Marc_Pref_Pat_Time(Sim/Nao),Adidas_Design,DryWorld_Design,...,TRP_score,RCP_score,SST_score,PER_score,AVP_score,SAT_score,PAO_score,FAT_score,LAT_score,LEC_score
0,1,1,25,Brasil,Campinas,2,5,1,6,99,...,-1.403417,-1.299009,-0.312851,-0.965156,-0.688917,-0.333053,-1.957198,-0.941725,-1.980388,-0.349405
1,2,1,9,Brasil,Goiania,1,1,2,7,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,13,Brasil,Belo_Horizonte,2,9,1,99,99,...,0.373639,0.054938,0.448074,0.529748,0.917137,0.827672,1.471288,1.020613,1.004458,1.034081
3,4,1,25,Brasil,Sao_Paulo,2,5,2,6,4,...,1.177616,0.879094,0.817992,0.828329,1.057682,1.554314,0.657640,0.723160,0.936833,0.333726
4,5,1,25,Brasil,Sao_Paulo,2,10,1,7,5,...,0.373639,-0.199645,0.102032,-1.085430,0.516827,-1.122820,-0.336414,1.333120,0.859230,1.136139


../.env/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


## Aplicar os testes de hipóteses

In [10]:
# cols = dados_testes.columns

In [11]:
# for c in cols:
#     print(c)

In [12]:
cols_groups = ['Genero', 'TV', 'PPV']
cols_score = ['Livro_score', 'IDF_score', 'TRP_score', 'RCP_score', 'SST_score', 'PER_score', 
              'AVP_score', 'SAT_score', 'PAO_score', 'FAT_score', 'LAT_score', 'LEC_score']

In [13]:
Results = []
for groups in cols_groups:
    for score in cols_score:
        
        #print(f"Testing for: {groups} and {score}")
        
        # filtrar na's
        dt = dados_testes[[groups, score]].dropna()
        dt = dt[dt[groups] != 99]
        
        # aplicar o teste
        test_result = pg.pairwise_tukey(data = dt,
                                        dv = score, 
                                        between = groups)
        test_result['group'] = groups
        test_result['score'] = score
        #print(test_result)
        Results.append(test_result)

In [14]:
results = pd.concat(Results)

In [15]:
results

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges,group,score
0,1,2,0.059015,-0.187343,0.246358,0.093207,2.643127,0.022716,0.262856,Genero,Livro_score
1,1,3,0.059015,-0.252431,0.311446,0.107280,2.903114,0.010556,0.332292,Genero,Livro_score
2,2,3,-0.187343,-0.252431,0.065088,0.133456,0.487714,0.864763,0.069248,Genero,Livro_score
0,1,2,0.152652,-0.583753,0.736405,0.092612,7.951488,0.001000,0.790260,Genero,IDF_score
1,1,3,0.152652,-0.281055,0.433708,0.081154,5.344276,0.001000,0.465445,Genero,IDF_score
2,2,3,-0.583753,-0.281055,-0.302698,0.113254,-2.672732,0.020864,-0.324231,Genero,IDF_score
0,1,2,0.052126,-0.116828,0.168954,0.095853,1.762634,0.182982,0.180069,Genero,TRP_score
1,1,3,0.052126,-0.230520,0.282646,0.095853,2.948746,0.009159,0.301242,Genero,TRP_score
2,2,3,-0.116828,-0.230520,0.113692,0.126403,0.899441,0.629204,0.120863,Genero,TRP_score
0,1,2,0.035918,-0.003550,0.039469,0.098208,0.401886,0.900000,0.041056,Genero,RCP_score


- Salvar resultados test t

In [16]:
# salvar os dados para analise das escalas
results.to_pickle('../data/dados_ttest.pkl')